In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classify-fruits-with-naive-bayes-from-scratch-b2/sample_submission.csv
/kaggle/input/classify-fruits-with-naive-bayes-from-scratch-b2/fruits_test.csv
/kaggle/input/classify-fruits-with-naive-bayes-from-scratch-b2/fruits_train.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/classify-fruits-with-naive-bayes-from-scratch-b2/fruits_train.csv",index_col=0)
print(train_data.head())
trd = train_data.values.tolist()
trd

    mass  width  height  label
Id                            
1    160    7.1     7.6      2
2    194    7.2    10.3      3
3    154    7.2     7.2      2
4    154    7.0     7.1      1
5    162    7.4     7.2      1


[[160.0, 7.1, 7.6, 2.0],
 [194.0, 7.2, 10.3, 3.0],
 [154.0, 7.2, 7.2, 2.0],
 [154.0, 7.0, 7.1, 1.0],
 [162.0, 7.4, 7.2, 1.0],
 [164.0, 7.2, 7.0, 2.0],
 [154.0, 7.1, 7.5, 2.0],
 [116.0, 6.1, 8.5, 3.0],
 [170.0, 7.6, 7.9, 1.0],
 [116.0, 5.9, 8.1, 3.0],
 [144.0, 6.8, 7.4, 2.0],
 [160.0, 7.5, 7.5, 1.0],
 [166.0, 6.9, 7.3, 1.0],
 [142.0, 7.6, 7.8, 2.0],
 [156.0, 7.4, 7.4, 1.0],
 [116.0, 6.0, 7.5, 3.0],
 [356.0, 9.2, 9.2, 2.0],
 [152.0, 6.5, 8.5, 3.0],
 [164.0, 7.3, 7.7, 1.0],
 [162.0, 7.5, 7.1, 1.0],
 [158.0, 7.1, 7.5, 2.0],
 [140.0, 7.3, 7.1, 1.0],
 [186.0, 7.2, 9.2, 3.0],
 [174.0, 7.3, 10.1, 3.0],
 [180.0, 8.0, 6.8, 1.0],
 [168.0, 7.5, 7.6, 1.0],
 [216.0, 7.3, 10.2, 3.0],
 [160.0, 7.0, 7.4, 2.0],
 [172.0, 7.1, 7.6, 1.0],
 [140.0, 6.7, 7.1, 2.0],
 [180.0, 7.6, 8.2, 2.0],
 [362.0, 9.6, 9.2, 2.0],
 [342.0, 9.0, 9.4, 2.0],
 [152.0, 7.6, 7.3, 1.0],
 [200.0, 7.3, 10.5, 3.0],
 [116.0, 6.3, 7.7, 3.0],
 [178.0, 7.1, 7.8, 1.0],
 [192.0, 8.4, 7.3, 1.0],
 [118.0, 5.9, 8.0, 3.0],
 [132.0, 5.8, 8.7, 3.

In [3]:
test_data = pd.read_csv("/kaggle/input/classify-fruits-with-naive-bayes-from-scratch-b2/fruits_test.csv",index_col=0)
print(test_data.head())
tsd = test_data.values.tolist()
tsd

    mass  width  height
Id                     
1    118    6.1     8.1
2    158    7.2     7.8
3    120    6.0     8.4
4    210    7.8     8.0
5    156    7.6     7.5


[[118.0, 6.1, 8.1],
 [158.0, 7.2, 7.8],
 [120.0, 6.0, 8.4],
 [210.0, 7.8, 8.0],
 [156.0, 7.6, 7.5],
 [176.0, 7.4, 7.2],
 [154.0, 7.3, 7.3],
 [196.0, 7.3, 9.7],
 [130.0, 6.0, 8.2],
 [150.0, 7.1, 7.9],
 [172.0, 7.4, 7.0],
 [156.0, 7.7, 7.1],
 [190.0, 7.5, 8.1],
 [204.0, 7.5, 9.2]]

In [4]:
def separate_by_class(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = []
        separated[class_value].append(vector)
    return separated

from math import sqrt
def mean(numbers):
    return sum(numbers) / float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x - avg) ** 2 for x in numbers]) / float(len(numbers) - 1)
    return sqrt(variance)

def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del summaries[-1]  
    return summaries

def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {}
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

from math import exp, sqrt, pi
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = {}
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2] / float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

In [5]:
model = summarize_by_class(trd)
model

{2.0: [(201.23076923076923, 87.4243995748649, 13),
  (7.630769230769231, 0.9741531496506894, 13),
  (7.884615384615386, 0.845425094105675, 13)],
 3.0: [(153.0, 38.76737147278741, 12),
  (6.566666666666666, 0.6400757530925302, 12),
  (8.941666666666666, 1.084148877174527, 12)],
 1.0: [(165.06666666666666, 12.668671520535929, 15),
  (7.439999999999999, 0.38321944477954517, 15),
  (7.379999999999998, 0.3028436654683168, 15)]}

In [6]:
my_pred=[]
for i in tsd:
    label=predict(model,i)
    my_pred.append(int(label))
my_pred

[3, 1, 3, 2, 1, 1, 1, 3, 3, 1, 1, 1, 2, 3]

In [7]:
my_sub = pd.DataFrame({'ID':test_data.index, 'Category':my_pred})
my_sub.to_csv('submission.csv',index=False)
my_sub

,ID,Category
0,1,3
1,2,1
2,3,3
3,4,2
4,5,1
5,6,1
6,7,1
7,8,3
8,9,3
9,10,1
